In [1]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = "/models/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
prompt_file_path = "/mnt/workspace/src/prompts/pyrep_quadcopter/real_planner_prompt.txt"
descriptions = "\nobjects = ['table', 'tree', 'sofa']\n# Query: fly to the table, then fly to the tree, and at last fly to the sofa"
with open(prompt_file_path, "r") as f:
    prompt = f.read().strip()
messages=[
    { 'role': 'user', 'content': f"continue below code, do not generate any non-cdoe message, only allowed python code\n\n{prompt}\n{descriptions}"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
import time
start_time = time.time()
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
end_time = time.time()
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))
print(f"Time taken: {end_time - start_time} seconds, Length: {len(result)} characters, ratio: {len(result)/(end_time - start_time)} characters per second.")
del outputs
del result
torch.cuda.empty_cache()

/usr/local/lib/python3.9/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


controlled_object = parse_query_obj('controlled object')
current_position = controlled_object.position
composer("move to 10cm above the table")
composer(f"move to 10cm above the tree")
composer(f"move to 10cm above the sofa")

# Execute the commands
execute()

Time taken: 4.173993825912476 seconds, Length: 260 characters, ratio: 62.29046108930491 characters per second.


In [32]:
import requests, json
messages = []
messages.append(
    {
        "role": "user",
        "content": "hello"
    }
)

payload = json.dumps(
    {
        "inputs": "write a python code about what is deep learning",
        "parameters": {
            "max_new_tokens": 512,
            "temperature": 0.1
        }
    }
)
headers = {"Content-Type": "application/json"}
try:
    response = requests.request(
        "POST",
        f'{"http://192.168.1.252:40906"}/generate',
        headers=headers,
        data=payload,
    )
    print(response.text)

    code_str = response.json()["result"]
except KeyError as e:
    print("KeyError:", e)
    print(response.content)
    print(payload)
    exit(1)
    # todo: if reach the max length of API limit, need to switch to a shorter version

        

{"generated_text":" and neural network.\n\nDeep learning is a subset of machine learning, which is essentially a neural network with three or more layers. These neural networks attempt to simulate the behavior of the human brain—albeit far from matching its ability—in order to \"learn\" from large amounts of data. While a neural network with a single layer can still make approximate predictions, additional hidden layers can help to optimize accuracy.\n\nNeural networks are composed of layers of nodes, or \"neurons,\" which are interconnected. Each connection, like the synapses in a brain, can transmit a signal from one neuron to another. The signals can be binary (representing 0 or 1) or continuous (representing any value between 0 and 1).\n\nThe \"learning\" in deep learning is achieved through a process called backpropagation, which adjusts the weights of the connections based on the difference between the predicted output and the actual output. This process is repeated many times, e